In [27]:
import pickle
import numpy as np
import os
import plotly
import pandas as pd

def getFamilyName(model_name):
    if 'densenet' in model_name.lower():
        model_family='DenseNet'
    elif 'inceptionresnet' in model_name.lower():
        model_family='InceptionResNet'
    elif 'efficientnet' in model_name.lower():
        model_family='EfficientNet'
    elif 'inception' in model_name.lower():
        model_family='Inception'
    elif 'mobilenet' in model_name.lower():
        model_family =  'MobileNet'
    elif 'nasnet' in model_name.lower():
        model_family =  'NASNet'
    elif 'resnet' in model_name.lower():
        model_family =  'ResNet'
    elif 'vgg' in model_name.lower():
        model_family = 'VGG'
    elif 'xception' in  model_name.lower():
        model_family = 'Xception'
    elif 'yolo' in model_name.lower():
        model_family = 'YOLO8'
    else :
        model_family = ''
    return model_family

def approx_USB_power_draw(row):
    if row['backend'] == 'tflite_edgetpu' or row['backend'] == 'NCS2':
        return row['power_draw'] + 25 * (row['running_time']/3600) # assume constant 25 watt
    else:
        return row['power_draw'] 


with open(os.getcwd()+'/result_databases/workstation_classification.pkl', 'rb') as f:
    data = pickle.load(f)
with open(os.getcwd()+'/result_databases/raspi_classification.pkl', 'rb') as f:
    data = data.append( pickle.load(f) )
with open(os.getcwd()+'/result_databases/workstation_segmentation.pkl', 'rb') as f:
    data = data.append( pickle.load(f) )
with open(os.getcwd()+'/result_databases/ls8nb010_class1.pkl', 'rb') as f:
    data = data.append( pickle.load(f).assign(architecture='Intel(R) Core(TM) i7-10610U') )
with open(os.getcwd()+'/result_databases/ls8nb010_segm1.pkl', 'rb') as f:
    data = data.append( pickle.load(f).assign(architecture='Intel(R) Core(TM) i7-10610U') )
with open(os.getcwd()+'/result_databases/ls8nb010_segm2.pkl', 'rb') as f:
    data = data.append( pickle.load(f).assign(architecture='Intel(R) Core(TM) i7-10610U') )


#print(data.columns)
#data = data.groupby(['model','backend','architecture']).mean().reset_index()

#Add Family of 
data['family']= data['model'].apply(lambda a : getFamilyName(a))
data['approx_USB_power_draw'] = data.apply(lambda row: approx_USB_power_draw(row), axis = 1)

#analyze_running_time = data[['model','backend','running_time']].dropna()#data[['model','backend','total_parameters', 'running_time','power_draw','accuracy','number_of_unmapped_operations']]

#analyze_running_time = analyze_running_time.pivot_table(index='model',columns = 'backend', values='running_time', aggfunc = 'mean')#set_index('model').wide_to_long()
#print('EDGETPU :' )
#print(np.mean(analyze_running_time['tflite_edgetpu']))
#print('CPU')
#print(np.mean(analyze_running_time['tf_cpu']))
#print('TFLITE')
#print(np.mean(analyze_running_time['tflite']))
#print('NCS2')
#print(np.mean(analyze_running_time['NCS2']))
easy_dict = {'Core i7-6700':'Workstation' ,'ARM Cortex-A72':'RasPi' ,'Intel(R) Core(TM) i7-10610U':'Laptop','tf_cpu':'','tflite_edgetpu':'EDGE TPU','NCS2':'NCS2','tflite':'TFLITE'}
color_dict = {'Workstation':{'EDGE TPU':'#c45f00','NCS2':'#5494DA','':'#b10000','TFLITE':'#b10000'},'Laptop':{'EDGE TPU':'#ff6f00','NCS2':'#86CEFA','':'#ff0000','TFLITE':'#ff0000'},'RasPi':{'EDGE TPU':'#633a00','NCS2':'#003396','TFLITE':'#300000','':'#300000'}}
fsize_dict = {'DenseNet121': 33.226576, 'DenseNet169': 58.593032, 'DenseNet201': 82.584656, 'EfficientNetB0': 21.856559999999998, 'EfficientNetB1': 32.186136, 'EfficientNetB2': 37.468056, 'EfficientNetB3': 50.136184, 'EfficientNetB4': 78.916208, 'EfficientNetB5': 123.524456, 'EfficientNetB6': 174.53435199999998, 'EfficientNetB7': 268.416968, 'InceptionResNetV2': 225.206864, 'InceptionV3': 108.98181699999999, 'MobileNetV2': 14.552912, 'MobileNetV3Large': 22.7354, 'MobileNetV3Small': 10.804839999999999, 'NASNetMobile': 23.037831999999998, 'QuickNet': 53.162167999999994, 'QuickNetLarge': 93.740032, 'QuickNetSmall': 50.845368, 'RegNetX32GF': 432.05466099999995, 'RegNetX400MF': 22.289129, 'RegNetX8GF': 158.813061, 'ResNet101': 179.68035999999998, 'ResNet152': 242.954816, 'ResNet50': 102.985688, 'ResNext101': 356.173021, 'ResNext50': 100.488385, 'VGG16': 553.491728, 'VGG19': 574.7381439999999, 'Xception': 91.973528,'yolov8s-seg':22544384,'yolov8n-seg':6501171.2,'yolov8m-seg':43725619.2}
data['architecture'] = data['architecture'].map(easy_dict)
data['backend'] = data['backend'].map(easy_dict)

data['fsize'] = data['model'].map(fsize_dict)

# add missing values that are equal across Models
for index, row in data.iterrows():
    current_model = row['model']
    if data.loc[data.model == current_model,'number_of_operations'].isnull:
        data.loc[data.model == current_model,'number_of_operations']=row['number_of_operations'] 
    data.loc[data.model == current_model,'number_of_unmapped_operations']=row['number_of_unmapped_operations'] 
    data.loc[data.model == current_model,'input_shape']=row['input_shape'] 
    data.loc[data.model == current_model,'total_parameters']=row['total_parameters'] 
    data.loc[data.model == current_model,'trainable_parameters']=row['trainable_parameters'] 
    data.loc[data.model == current_model,'non_trainable_parameters']=row['non_trainable_parameters'] 

data["number_of_operations"]=pd.to_numeric(data["number_of_operations"])
data["number_of_unmapped_operations"]=pd.to_numeric(data["number_of_unmapped_operations"])


data["total_parameters"]=pd.to_numeric(data["total_parameters"])

data["trainable_parameters"]=pd.to_numeric(data["trainable_parameters"])
data["non_trainable_parameters"]=pd.to_numeric(data["non_trainable_parameters"])
data["power_draw"] = data["power_draw"]*3600
data["approx_USB_power_draw"] = data["approx_USB_power_draw"]*3600


/tmp/ipykernel_942085/1539661992.py:42: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_942085/1539661992.py:44: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_942085/1539661992.py:46: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_942085/1539661992.py:48: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_942085/1539661992.py:50: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [28]:
print(data['backend'].unique())
print(data.columns)

['NCS2' '' 'EDGE TPU' nan 'TFLITE']
Index(['log_name', 'task', 'dataset', 'model', 'backend', 'software',
       'architecture', 'number_of_operations', 'number_of_unmapped_operations',
       'input_shape', 'total_parameters', 'trainable_parameters',
       'non_trainable_parameters', 'running_time', 'power_draw',
       'validation_size', 'batch_size', 'accuracy_k1', 'accuracy_k3',
       'accuracy_k5', 'accuracy_k10', 'precision_B', 'recall_B', 'mAP50_B',
       'mAP50_95_B', 'precision_M', 'recall_M', 'mAP50_M', 'mAP50_95_M',
       'family', 'approx_USB_power_draw', 'fsize'],
      dtype='object')


In [29]:
data.to_csv('whole_database.csv')

# Performance Comparison:
Bar Chart: Accuracy Comparison


In [30]:

import plotly.graph_objects as go
import math
classification_data = data[data['log_name'].str.contains("classification")].reset_index()
segmentation_data = data[data['log_name'].str.contains("segmentation")].reset_index()
classification_data_grouped = classification_data.groupby(['model','backend','architecture']).mean().reset_index()
classification_data_grouped=classification_data_grouped[['model','backend','architecture','accuracy_k1']]
classification_data_grouped.dropna(subset=['accuracy_k1'], inplace=True)
classification_data_grouped.to_csv('classification_data.csv')
from plotly.subplots import make_subplots
fig = make_subplots(rows=7, cols=2, start_cell="top-left", subplot_titles=classification_data_grouped['model'].unique(),shared_yaxes=True)
for i in range(len(classification_data_grouped['model'].unique())):
    current_model = classification_data_grouped['model'].unique()[i]
    #print(current_model)
    current_df = classification_data_grouped[classification_data_grouped['model']==current_model].sort_values('architecture')
    x = current_df['backend'] + ' ' + current_df['architecture']
    y = current_df['accuracy_k1']
    #print(math.floor(i/2)) 
    #print((i%2)+1)
    fig.add_trace(go.Bar(x=x, y=y),row=math.floor(i/2)+1, col=round(i%2)+1)
fig.update_layout(height=1700, width=1000,showlegend=False,
                  title_text="Accuracies of Different Models on Different Environments")
fig.show()


In [31]:
segmentation_data = data[data['log_name'].str.contains("segmentation")].reset_index()
segmentation_data_grouped = segmentation_data.groupby(['model','backend','architecture']).mean().reset_index()
segmentation_data_grouped=segmentation_data_grouped[['model','backend','architecture','precision_B']]
segmentation_data_grouped.dropna(subset=['precision_B'], inplace=True)
segmentation_data_grouped.to_csv('classification_data.csv')
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=3, start_cell="top-left", subplot_titles=segmentation_data_grouped['model'].unique(),shared_yaxes=True)
for i in range(len(segmentation_data_grouped['model'].unique())):
    current_model = segmentation_data_grouped['model'].unique()[i]
    #print(current_model)
    current_df = segmentation_data_grouped[segmentation_data_grouped['model']==current_model].sort_values('architecture')
    x = current_df['backend'] + ' ' + current_df['architecture']
    y = current_df['precision_B']
    display_color_dict =current_df['architecture'].map(color_dict)
    #colors = []
    #for it in range(len(display_color_dict)):
        #colors.append(display_color_dict[it][current_df['backend'][it]])
    # current_df['backend'].map(
   
   
    fig.add_trace(go.Bar(x=x, y=y),row=1, col=i+1)
fig.update_layout(height=300, width=1000,
                  title_text="Multiple Subplots with Titles")
fig.show()


In [32]:
! pip show pandas

Name: pandas
Version: 1.4.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /opt/anaconda3/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: datashader, holoviews, hvplot, seaborn, statsmodels, ultralytics, xarray


In [33]:
classification_data_grouped_consumption = classification_data.groupby(['model','backend','architecture']).mean().reset_index()
#classification_data['Coral_Energy_Saved'] 
fig = go.Figure()

x = classification_data_grouped_consumption['model'].unique()
for curr_architecture in classification_data_grouped_consumption['architecture'].unique():
    for curr_backend in classification_data_grouped_consumption['backend'].unique():
        curr_df = classification_data_grouped_consumption[(classification_data_grouped_consumption['backend']==curr_backend )& (classification_data_grouped_consumption['architecture']==curr_architecture )]
        color = color_dict[curr_architecture][curr_backend]
        y = []
        for model_idx in range(len(x)):
            y.append(np.nan)
            y[model_idx]=curr_df[curr_df['model']== x[model_idx]]['running_time']
        print(x)
        print(y)
        fig.add_trace(go.Scatter( x=x,y=y,marker=dict(color=color_dict[curr_architecture][curr_backend])))
fig.update_layout(height=300, width=5000,
                  title_text="Multiple Subplots with Titles")


fig.show()

['DenseNet121' 'DenseNet169' 'DenseNet201' 'EfficientNetB0' 'InceptionV3'
 'MobileNet' 'MobileNetV2' 'NASNetMobile' 'ResNet101' 'ResNet101V2'
 'ResNet50' 'ResNet50V2' 'VGG16' 'Xception']
[0    0.048528
Name: running_time, dtype: float64, 8    0.053684
Name: running_time, dtype: float64, 16    0.074402
Name: running_time, dtype: float64, 24    0.021208
Name: running_time, dtype: float64, 30    0.068616
Name: running_time, dtype: float64, 37    0.011292
Name: running_time, dtype: float64, 45    0.010622
Name: running_time, dtype: float64, 53    0.019178
Name: running_time, dtype: float64, 61    0.095713
Name: running_time, dtype: float64, 69    0.070591
Name: running_time, dtype: float64, 77    0.042919
Name: running_time, dtype: float64, 85    0.045265
Name: running_time, dtype: float64, 93    0.165157
Name: running_time, dtype: float64, 98    0.119467
Name: running_time, dtype: float64]
['DenseNet121' 'DenseNet169' 'DenseNet201' 'EfficientNetB0' 'InceptionV3'
 'MobileNet' 'MobileNetV2'

# Improvements in Energy Consumption and Duration

In [34]:
#classification_data_grouped_consumption
#for curr_architecture in classification_data_grouped_consumption['architecture'].unique():
  #  for curr_backend in classification_data_grouped_consumption['backend'].unique():
        

In [35]:
segmentation_data.groupby(['model','backend','architecture']).agg(['mean','var']).reset_index()

/tmp/ipykernel_942085/4038406879.py:1: FutureWarning:

['log_name', 'task', 'dataset', 'software', 'input_shape', 'family'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



model   backend architecture      index              \
                                             mean         var   
0   yolov8m-seg                 Laptop   4.500000    0.333333   
1   yolov8m-seg            Workstation  11.333333   66.333333   
2   yolov8m-seg  EDGE TPU       Laptop   6.500000    0.333333   
3   yolov8m-seg  EDGE TPU  Workstation  16.666667  140.333333   
4   yolov8m-seg      NCS2  Workstation   9.000000    9.000000   
5   yolov8n-seg                 Laptop   0.500000    0.333333   
6   yolov8n-seg            Workstation  10.000000   76.000000   
7   yolov8n-seg  EDGE TPU       Laptop   2.500000    0.333333   
8   yolov8n-seg  EDGE TPU  Workstation  14.666667  140.333333   
9   yolov8n-seg      NCS2  Workstation   8.000000    9.000000   
10  yolov8s-seg                 Laptop   8.500000    0.333333   
11  yolov8s-seg            Workstation  19.000000    1.000000   
12  yolov8s-seg  EDGE TPU       Laptop  10.500000    0.333333   
13  yolov8s-seg  EDGE TPU  Workstation  18.333333  154.333333   
14  yolov8s-seg      NCS2  Workstation  10.000000    9.000000   

   number_of_operations      number_of_unmapped_operations       \
                   mean  var                          mean  var   
0                 360.0  0.0                         210.0  0.0   
1                 360.0  0.0                         210.0  0.0   
2                 360.0  0.0                         210.0  0.0   
3                 360.0  0.0                         210.0  0.0   
4                 360.0  0.0                         210.0  0.0   
5                 294.0  0.0                         186.0  0.0   
6                 294.0  0.0                         186.0  0.0   
7                 294.0  0.0                         186.0  0.0   
8                 294.0  0.0                         186.0  0.0   
9                 294.0  0.0                         186.0  0.0   
10                293.0  0.0                         185.0  0.0   
11                293.0  0.0                         185.0  0.0   
12                293.0  0.0                         185.0  0.0   
13                293.0  0.0                         185.0  0.0   
14                293.0  0.0                         185.0  0.0   

   total_parameters  ...  recall_M             mAP50_M           mAP50_95_M  \
               mean  ...      mean       var      mean       var       mean   
0        27285968.0  ...  0.621439  0.002132  0.680676  0.003378   0.452296   
1        27285968.0  ...  0.581448  0.000000  0.630327  0.000000   0.409402   
2        27285968.0  ...  0.526898  0.000176  0.561189  0.004235   0.356065   
3        27285968.0  ...  0.515461  0.000000  0.504972  0.000000   0.312531   
4        27285968.0  ...  0.583006  0.000000  0.630519  0.000000   0.409483   
5         3409968.0  ...  0.470287  0.000277  0.521664  0.001392   0.334689   
6         3409968.0  ...  0.455898  0.000000  0.489357  0.000000   0.304266   
7         3409968.0  ...  0.393836  0.000588  0.384413  0.001345   0.234055   
8         3409968.0  ...  0.372768  0.000000  0.352616  0.000000   0.208577   
9         3409968.0  ...  0.455924  0.000000  0.489546  0.000000   0.304297   
10       11821056.0  ...  0.000000  0.000000  0.000000  0.000000   0.000000   
11       11821056.0  ...  0.531394  0.000000  0.576948  0.000000   0.369698   
12       11821056.0  ...  0.464117  0.001368  0.498917  0.004927   0.308675   
13       11821056.0  ...  0.432066  0.000000  0.438112  0.000000   0.263960   
14       11821056.0  ...  0.530761  0.000000  0.576935  0.000000   0.369717   

             approx_USB_power_draw                     fsize       
         var                  mean           var        mean  var  
0   0.002455          69097.343806  2.042701e+07  43725619.2  0.0  
1   0.000000         182820.590998  1.014977e+07  43725619.2  0.0  
2   0.002534         146266.090344  1.919968e+07  43725619.2  0.0  
3   0.000000         183243.904125  5.545647e+05  43725619.2  0.0  
4   0.000000

In [36]:
#classification_data.groupby(['model','backend','architecture']).agg(['mean','var']).reset_index()[]
print(classification_data[['number_of_operations','model']])

     number_of_operations         model
0                    75.0      ResNet50
1                    75.0      ResNet50
2                    75.0      ResNet50
3                   431.0   DenseNet169
4                   431.0   DenseNet169
..                    ...           ...
289                  69.0   MobileNetV2
290                 583.0  NASNetMobile
291                 583.0  NASNetMobile
292                 143.0     ResNet101
293                 143.0     ResNet101

[294 rows x 2 columns]


In [41]:
export_class_df = classification_data.drop(['index','precision_B', 'recall_B', 'mAP50_B','mAP50_95_B', 'precision_M', 'recall_M', 'mAP50_M', 'mAP50_95_M','family'],axis=1)
export_class_df = export_class_df.groupby(['model','backend','architecture']).mean().reset_index()
export_class_df['task'] = 'infer'
export_class_df['dataset'] = 'imagenet'
export_class_df['environment'] = export_class_df['backend'] + export_class_df['architecture']
export_class_df = export_class_df.drop(export_class_df[export_class_df['model'] == 'VGG16'].index)
export_class_df = export_class_df.drop(export_class_df[export_class_df['architecture'] == 'Laptop'].index)

export_class_df.to_pickle(os.path.join(os.getcwd(),'classification_database/database.pkl')) 
export_seg_df = segmentation_data.drop(['index','accuracy_k1', 'accuracy_k3',
       'accuracy_k5', 'accuracy_k10','family'],axis=1)

export_seg_df = export_seg_df.groupby(['model','backend','architecture']).mean().reset_index()
export_seg_df['task'] = 'infer'
export_seg_df['dataset'] = 'coco'
export_seg_df['environment'] = export_seg_df['backend'] + export_seg_df['architecture']

export_seg_df.to_pickle(os.path.join(os.getcwd(),'segmentation_database/database.pkl'))

IN Main

In [ ]:
import plotly.graph_objects as go
def prop_dict_to_val(df, key='value'):
    return df.applymap(lambda val: val[key] if isinstance(val, dict) and key in val else val)
from mlprops.util import read_json, lookup_meta, identify_all_correlations
from mlprops.index_and_rate import prop_dict_to_val
from mlprops.load_experiment_logs import find_sub_db
from mlprops.elex.util import ENV_SYMBOLS, RATING_COLORS, RATING_COLOR_SCALE, RATING_COLOR_SCALE_REV
from mlprops.elex.graphs import assemble_scatter_data, create_scatter_graph, add_rating_background
PLOT_WIDTH = 1000
PLOT_HEIGHT = PLOT_WIDTH // 3
COLORS = ['#009ee3', '#983082', '#ffbc29', '#35cdb4', '#e82e82', '#59bdf7', '#ec6469', '#706f6f', '#4a4ad8', '#0c122b', '#ffffff']

SEL_DS_TASK = {
    'ImageNetEff': ('imagenet', 'infer'),
    'CocoEff': ('coco', 'infer'),
}


In [38]:
for name, (db, meta, metrics, xdef, ydef, bounds, _, _) in databases.items():
        ds, task = SEL_DS_TASK[name]
        db = prop_dict_to_val(db, 'index')
        ds_name = lookup_meta(meta, ds, subdict='dataset')

        worst = db.sort_values('compound_index').iloc[0]
        best = db.sort_values('compound_index').iloc[-1]
        fig = go.Figure()
        for model, col, m_str in zip([best, worst], [RATING_COLORS[0], RATING_COLORS[4]], ['Best', 'Worst']):
            mod_name = lookup_meta(meta, model['model'], 'short', 'model')[:18]
            print(metrics)
            metr_names = [lookup_meta(meta, metr, 'shortname', 'properties') for metr in metrics[(ds, task)]]
            fig.add_trace(go.Scatterpolar(
                r=[model[col] for col in metrics[(ds, task)]], line={'color': col},
                theta=metr_names, fill='toself', name=f'{mod_name} ({m_str}): {model["compound_index"]:4.2f}'
            ))
        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True)), width=PLOT_WIDTH*0.25, height=PLOT_HEIGHT, title_y=1.0, title_x=0.5, title_text=ds_name,
            legend=dict( yanchor="bottom", y=1.06, xanchor="center", x=0.5), margin={'l': 30, 'r': 30, 'b': 15, 't': 70}
        )
        fig.write_image(f'true_best_{name}.pdf')


NameError: name 'db' is not defined